### Sucky Policy

Agent takes care of macro actions and paths, and depleted battery
First try only one piece of gold

Best Move:
*  SUCK if at (r,c) and dirt at (r,c)
*  UNLOAD if at (1,1) and have gold
*  GOTO (1,1) if have gold
*  MINE if at (r,c) and gold at (r,c)
*  GOTO (r,c) gold at (r,c)



In [ ]:
##  Build recon for a world
from constants import EAST

recon = {
    'height': 5,
    'width': 5,
    'position': (1,1),
    'heading': EAST,
    'gold': [(3,3)],
    'walls': [(0,0), (0,1), (0,2), (0,3), (0,4),
             (1,0), (1,4),
             (2,0), (2,4),
             (3,0), (3,4),
             (4,0), (4,1), (4,2), (4,3), (4,4),
             (2,2), (3,1)],
    'dirt': [(1,2)]
}

In [68]:
from pyDatalog import pyDatalog
from pyDatalog.pyDatalog import assert_fact, retract_fact, ask, create_terms, clear

from constants import WALL, GOLD, DIRT, CLEAN
from constants import ACTION_FORWARD, ACTION_TURN_LEFT, ACTION_TURN_RIGHT, ACTION_MINE_GOLD, ACTION_UNLOAD_GOLD

create_terms('location, heading, square_state, num_gold, last_action')

def ask_pred(pred):
    return ask(f"{pred}(_)").answers[0][0]

def ask_square_state(sq):
    a = ask(f"square_state(({sq}, _))")
    if a == None:
        return None
    else:
        return a.answers[0][0]

def init_world_model():
    clear()
    assert_fact('num_gold', 0)
    assert_fact('last_action', None)
            
def tell_recon(recon):
    assert_fact('location', recon['position'])
    assert_fact('heading', recon['heading'])
    if 'walls' in recon:
        for square in recon['walls']:
            assert_fact('square_state', (square, WALL))
    if 'gold' in recon:
        for square in recon['gold']:
            assert_fact('square_state', (square, GOLD))
    
def tell_last_action(action):
    assert_fact('last_action', action)
        
def tell_percepts(percepts):
    dirt = percepts.attributes['dirt']
    bump = percepts.attributes['bump']
    last_action = ask_pred('last_action')
    location = ask_pred('location')
    heading = ask_pred('heading')
    num_gold = ask_pred('num_gold')
    if last_action == ACTION_FORWARD and bump:
        assert_fact('square_state', (square_at_heading(location, heading), WALL))
    elif last_action == ACTION_FORWARD and not bump:
        newsquare = square_at_heading(location, heading)
        assert_fact('location', newsquare)
        assert_fact('square_state', (newsquare, DIRT if dirt else CLEAN))
    elif last_action == ACTION_TURN_LEFT:
        assert_fact('heading', heading_in_direction(heading, LEFT))
    elif last_action == ACTION_TURN_RIGHT:
        assert_fact('heading', heading_in_direction(heading, RIGHT))
    elif last_action == ACTION_MINE_GOLD and ask_square_state(location) == GOLD:
        assert_fact('square_state', (location, CLEAN))
        assert_fact('num_gold', num_gold+1)
    elif last_action == ACTION_UNLOAD_GOLD and location == (1,1):
        assert_fact('num_gold', num_gold-1)
    # For first step, last_action is None but want to set DIRT anyway
    if dirt:
        assert_fact('square_state', (location, DIRT))
    else:
        if ask_square_state(location) != GOLD:
            assert_fact('square_state', (location, CLEAN))


In [57]:
init_world_model()

In [58]:
##  Build recon for a world
from constants import EAST

recon = {
    'height': 5,
    'width': 5,
    'position': (1,1),
    'heading': EAST,
    'gold': [(3,3)],
    'walls': [(0,0), (0,1), (0,2), (0,3), (0,4),
             (1,0), (1,4),
             (2,0), (2,4),
             (3,0), (3,4),
             (4,0), (4,1), (4,2), (4,3), (4,4),
             (2,2), (3,1)],
    'dirt': [(1,2)]
}
tell_recon(recon)

In [9]:
pyDatalog.ask('location(X)').answers

[((1, 1),)]

In [10]:
ask_pred('heading')

'east'

In [ ]:
tell_recon(recon)

In [11]:
ask(f"square_state((X, gold))").answers

[((3, 3),)]

In [12]:
ask(f"square_state((X, wall))").answers

[((3, 1),),
 ((2, 2),),
 ((4, 4),),
 ((4, 3),),
 ((4, 2),),
 ((4, 1),),
 ((4, 0),),
 ((3, 4),),
 ((3, 0),),
 ((2, 4),),
 ((2, 0),),
 ((1, 4),),
 ((1, 0),),
 ((0, 4),),
 ((0, 3),),
 ((0, 2),),
 ((0, 1),),
 ((0, 0),)]

SUCK if at (r,c) and dirt at (r,c)
UNLOAD if at (1,1) and have gold
GOTO (1,1) if have gold
MINE if at (r,c) and gold at (r,c)
GOTO (r,c) gold at (r,c)

In [83]:
from constants import ACTION_SUCK, ACTION_STOP
def init_rules():
    create_terms('best_action', 'S', 'T')
    best_action(ACTION_SUCK) <= location(S) & square_state((S, DIRT)) 
    best_action(ACTION_UNLOAD_GOLD) <= location(S) & (S==(1,1)) & num_gold(T) & (T > 0)
    best_action(ACTION_MINE_GOLD) <= location(S) &  square_state((S, GOLD)) & num_gold(T) & (T < 2)
    best_action(("GOTO", S)) <= square_state((S, GOLD)) & num_gold(T) & (T < 2)
    best_action(("GOTO", (1,1))) <= num_gold(T) & (T > 1)
    +best_action(None)

In [84]:
class Percept:
    def __init__(self, d):
        self.attributes = d

In [85]:
init_world_model()
tell_recon(recon)
tell_percepts(Percept({"dirt": False, "bump": False}))
init_rules()
+num_gold(1)

In [86]:
best_action(S)

[(None,), (('GOTO', (3, 3)),), ('unloadgold',)]

In [81]:
num_gold(S)

[(1,), (0,)]

In [31]:
square_state(((1,1), T))

[]

In [19]:
square_state(((1,1), T))

[]

In [ ]:
tell_recon(recon)

In [ ]:
location(X)

In [ ]:
replace_location(2,3)

In [ ]:
pyDatalog.ask('location(X,Y)').answers

In [ ]:
tell_recon(recon)

In [ ]:
def dostuff():
    assert_fact("foo", 1)
    assert_fact("foo", 2)
    assert_fact("foo", 3)

In [ ]:
clear()
create_terms('foo', 'bar')
dostuff()

In [ ]:
foo(X)

In [ ]:
clear()
create_terms('square_state')
+square_state(((1,2), "foo"))


In [ ]:
ask_square_state((1,3))

In [21]:
create_terms("a", "b", "c")
+a(1)
+a(2)
+a(3)
+b(2)
+b(3)
+b(4)
+c(3)
+c(4)
+c(5)

In [23]:
create_terms("X")
a(X)

[(3,), (2,), (1,)]

In [26]:
a(X) & b(X)

[(3,), (2,)]

In [25]:
b(X)

[(4,), (3,), (2,)]